<center> <img src="https://storage.cloud.google.com/arize-assets/phoenix/assets/phoenix-logo-light.svg" width="300"/> </center>

# <center>Getting Started with Hosted Phoenix

This guide demonstrates how to use hosted phoenix, our solution for AI developers to trace and evaluate their LLM applications without setting up infrastructure. Read more about Phoenix [here](https://docs.arize.com/phoenix/).

ℹ️ This notebook requires an OpenAI API key


## Step 1: Install Dependencies 📚
Let's get the notebook setup with dependencies. It does NOT require installing arize-phoenix to log traces, as we are compliant with OpenTelemetry tracing.

In [ ]:
!pip install arize-otel openinference-instrumentation-openai openai

In [ ]:
import os
from getpass import getpass

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")

os.environ["OPENAI_API_KEY"] = openai_api_key

🔑 Enter your OpenAI API key: ··········


In [ ]:
import os
from getpass import getpass

if not (PHOENIX_API_KEY := os.getenv("PHOENIX_API_KEY")):
    PHOENIX_API_KEY = getpass("🔑 Enter your Phoenix API key: ")

os.environ["PHOENIX_API_KEY"] = PHOENIX_API_KEY

🔑 Enter your Phoenix API key: ··········


## Step 2: Setup Tracing
Let's send a trace to Hosted Phoenix! Notce the key lines below using `OTEL_EXPORTER_OTLP_HEADERS` and `app.phoenix.arize.com/v1/traces`

In [ ]:
from arize_otel import Endpoints, register_otel
from openinference.instrumentation.openai import OpenAIInstrumentor

# Setup OTEL tracing for hosted Phoenix
register_otel(endpoints=[Endpoints.HOSTED_PHOENIX], api_key=PHOENIX_API_KEY)

# Turn on instrumentation for OpenAI
OpenAIInstrumentor().instrument()

Send query to Hosted Phoenix


In [ ]:
import openai

openai_client = openai.OpenAI()
response = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Write a haiku."}],
    max_tokens=20,
)
print(response.choices[0].message.content)

Whispers in the wind
Leaves ruffle with gentle touch
Nature's soft embrace


## Step 3: Use the Phoenix SDK
First, install the right python packages to setup phoenix.


In [ ]:
!pip install "arize-phoenix[evals]"

Set the `PHOENIX_CLIENT_HEADERS` and the `PHOENIX_COLLECTOR_ENDPOINT` to the appropriate values to start using hosted Phoenix!

In [ ]:
os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com/"

from datetime import datetime, timedelta

import phoenix as px

# Initiate Phoenix client
px_client = px.Client()

# Get spans from the last 7 days only
start = datetime.now() - timedelta(days=7)

# Get spans to exclude the last 24 hours
end = datetime.now() - timedelta(days=0)

phoenix_df = px_client.query_spans(start_time=start, end_time=end)
print(phoenix_df.head())
px_client.get_spans_dataframe(project_name="rechat", root_spans_only=True)

                            name span_kind parent_id  \
context.span_id                                        
30f18366408abf1f  ChatCompletion       LLM      None   

                                       start_time  \
context.span_id                                     
30f18366408abf1f 2024-07-25 21:23:21.506330+00:00   

                                         end_time status_code status_message  \
context.span_id                                                                
30f18366408abf1f 2024-07-25 21:23:22.170959+00:00          OK                  

                 events   context.span_id                  context.trace_id  \
context.span_id                                                               
30f18366408abf1f     []  30f18366408abf1f  e1a9815516cb9d2c618b03d975fda1d5   

                  ... attributes.output.mime_type  attributes.input.mime_type  \
context.span_id   ...                                                           
30f18366408abf1f  ...         

,name,span_kind,parent_id,start_time,end_time,status_code,status_message,events,context.span_id,context.trace_id
context.span_id,,,,,,,,,,


Use our latest and greatest features around datasets on the latest Phoenix version

In [ ]:
import pandas as pd
import phoenix as px

df = pd.DataFrame(
    [
        {
            "question": "What is Paul Graham known for?",
            "answer": "Co-founding Y Combinator and writing on startups and techology.",
            "metadata": {"topic": "tech"},
        }
    ]
)
phoenix_client = px.Client()
dataset = phoenix_client.upload_dataset(
    dataframe=df,
    dataset_name="test-dataset-2",
    input_keys=["question"],
    output_keys=["answer"],
    metadata_keys=["metadata"],
)

📤 Uploading dataset...
💾 Examples uploaded: https://app.phoenix.arize.com/datasets/RGF0YXNldDoy/examples
🗄️ Dataset version ID: RGF0YXNldFZlcnNpb246Mg==
